# Chap05 变形

In [1]:
import numpy as np
import pandas as pd

## 长宽表的变形
1. 长表：信息比较详细的表
2. 宽表：信息比较汇总的表
3. 信息等价，只是数值的呈现方式不同
### `pivot`长表变宽表的函数
1. 三要素：变形后的行索引`index`、需要转到列索引的列`columns`、列和行索引对应的数值`values`
2. 新生成表的列索引是**columns**对应列的唯一值，新表的行索引是**index**对应列的唯一值，**values**对应了想要展示的数值列
3. pivot进行变形操作需要满足唯一性的要求，即新表中的行索引对应的列和列索引对应的列的组合必须唯一
4. 三个参数允许被设置为列表
### `pivot_table`
1. 如果不满足唯一性条件，必须通过聚合操作使得相同行列组合对应的多个值变为一个值
2. `aggfunc`参数指定使用的聚合函数，也可以传入以序列为输入标量为输出的聚合函数实现自定义操作
3. 还具有**边际汇总的功能**，可以通过设置`margins=True`实现，其中边际的聚合方式与`aggfunc`中给出的聚合方法一致
### `melt`宽表变长表的函数
1. `id_vars`长表的类似id列名(来自宽表的列名)
2. `value_vars`宽表的值所在的列名(来自宽表的列名)
3. `var_name`宽表的值所在的列转化为长表后变成值，该列的列名(自定义)
4. `value_name`值的列名(自定义)
5. 局限：在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，如果列中包含了交叉类别，无法把values_name扩充为多列
### `wide_to_long`解决`melt`的局限
1. `pd.wide_to_long(df,stubnames,i,j,sep,suffix)`
   - `df`待操作的数据集
   - `stubnames`值所在的列名，需自定义，相当于`melt`的**value_name**
   - `i`长表的类似id列名，来自宽表的列名，相当于`melt`的**id_vars**
   - `j`压缩到行的变量名含义，主要是为了展示不同类别的含义，需自定义，相当于`melt`的**var_name**
   - `sep`分隔符
   - `suffix`正则后缀

In [2]:
# 长表
pd.DataFrame({'Gender':['F','F','M','M'], 'Height':[163,160,175,180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
# 宽表
pd.DataFrame({'Height:F':[163,160], 'Height:M':[175,180]})

,Height:F,Height:M
0,163,175
1,160,180


In [4]:
df = pd.DataFrame({'Class': [1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [8]:
df.pivot(index=['Class','Name'], columns='Subject', values='Grade')

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,75
2,Si Li,90,85


In [2]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [3]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [4]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x:x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [5]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


In [8]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80,90],
                   'Math':[80,75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [9]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese','Math'],
                    var_name = 'Subject',
                    value_name='Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


In [2]:
df = pd.DataFrame({'Class':[1,2], 'Name':['San Zhang','Si Li'],
                   'Chinese_Mid':[80,75],'Math_Mid':[90,85],
                   'Chinese_Final':[80,75],'Math_Final':[90,85]})
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,85,75,85


In [3]:
pd.wide_to_long(df,
                stubnames=['Chinese','Math'],
                i = ['Class','Name'],
                j = 'Examination',
                sep = '_',
                suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

## 索引的变形
### unstack行索引转为列索引
1. 主要参数是移动的层号，默认转化最内层，移动到列索引的最内层，同时支持同时转化多个层(列表的形式)
2. 必须保证**被转为列索引的行索引层**和**被保留的行索引层**构成的组合是唯一的
### stack列索引转为行索引
1. 用法与`unstack`相似
### 聚合与变形的关系
1. 聚合后的values的个数产生了变化，变形前后不会带来values个数的改变

In [12]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A','cat','big'),
                                    ('A','dog','small'),
                                    ('B','cat','big'),
                                    ('B','dog','small')]),
                  columns=['col_1','col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [5]:
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [7]:
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

In [13]:
df.unstack([0,2,1])

col_1  A  big    cat    1.0
          small  dog    1.0
       B  big    cat    1.0
          small  dog    1.0
col_2  A  big    cat    1.0
          small  dog    1.0
       B  big    cat    1.0
          small  dog    1.0
dtype: float64

In [10]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A','cat','big'),
                                    ('A','cat','big'),
                                    ('B','cat','big'),
                                    ('B','dog','small')]),
                  columns=['col_1','col_2'])
df
# 将最内层的行索引转化为列索引时会破坏唯一性

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [15]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A','cat','big'),
                                    ('A','dog','small'),
                                    ('B','cat','big'),
                                    ('B','dog','small')]),
                  columns=['index_1','index_2']).T
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [18]:
df.stack([0,1])

big  small
index_1 A cat  1.0    NaN
          dog  NaN    1.0
        B cat  1.0    NaN
          dog  NaN    1.0
index_2 A cat  1.0    NaN
          dog  NaN    1.0
        B cat  1.0    NaN
          dog  NaN    1.0

## 其他变形函数
### crosstab
1. 该函数所能实现的功能`pivot_table`都能完成
   - 区别：`crosstab`的对应位置传入的是具体的序列，`pivot_table`传入的是被调用表对应的名字
2. 默认状态下，`crosstab`统计元素组合出现的频数
   - 除了默认状态下的count统计，所有聚合字符串和**返回标量**的自定义函数都是可用的
### explode
1. 能够对某一列的元素进行纵向的展开，被展开的单元格必须存储`list/tuple/Series/np.ndarray`中的一种类型
### get_dummies
1. 用于特征构建的重要函数之一，其作用是把类别特征转化为指示变量

In [19]:
df = pd.read_csv('./data/learn_pandas.csv')
pd.crosstab(index = df.School, columns = df.Transfer)

Transfer,N,Y
School,,
Fudan University,38,1
Peking University,28,2
Shanghai Jiao Tong University,53,0
Tsinghua University,62,4


In [20]:
pd.crosstab(index = df.School, columns=df.Transfer, values=df.Height, aggfunc='mean')

Transfer,N,Y
School,,
Fudan University,162.043750,177.20
Peking University,163.429630,162.40
Shanghai Jiao Tong University,163.953846,NaN
Tsinghua University,163.253571,164.55


In [21]:
df_ex = pd.DataFrame({'A':[[1,2], 'my_str', {1,2}, pd.Series([3,4])],
                      'B':1})

In [22]:
df_ex.explode('A')

,A,B
0,1,1
0,2,1
1,my_str,1
2,1,1
2,2,1
3,3,1
3,4,1


In [23]:
pd.get_dummies(df.Grade)

,Freshman,Junior,Senior,Sophomore
0,1,0,0,0
1,1,0,0,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
195,0,1,0,0
196,0,0,1,0
197,0,0,1,0
198,0,0,1,0
